In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import os
import sys
print(os.listdir('/content/gdrive/My Drive/NLP-202-A1-Code'))
sys.path.append('/content/gdrive/My Drive/NLP-202-A1-Code')

In [21]:
def read_gazetteer_file(file_path):
    gazetteer = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.strip():  # Ensure the line is not empty
                parts = line.strip().split(' ', 1)  # Split the line into two parts
                if len(parts) == 2:
                    tag, entities = parts
                    entities_list=entities.split(' ')
                    for entity in entities_list:
                        if entity not in gazetteer:
                            gazetteer[entity] = []
                        gazetteer[entity].append(tag)
                else:
                    print(f"Invalid line format: {line}")
    return gazetteer

# Example usage
file_path = './gazetteer.txt'  # Change to your actual file path
gazetteer = read_gazetteer_file(file_path)
# print(gazetteer)


In [20]:
import random
from conlleval import evaluate as conllevaluate
import numpy as np


def decode(input_length, tagset, score):
    """
    Compute the highest scoring sequence according to the scoring function.
    :param input_length: int. number of tokens in the input including <START> and <STOP>
    :param tagset: Array of strings, which are the possible tags.  Does not have <START>, <STOP>
    :param score: function from current_tag (string), previous_tag (string), i (int) to the score.  i=0 points to
        <START> and i=1 points to the first token. i=input_length-1 points to <STOP>
    :return: Array strings of length input_length, which is the highest scoring tag sequence including <START> and <STOP>
    """
    # Look at the function compute_score for an example of how the tag sequence should be scored
    
    # Initialize Viterbi matrix and back pointers
    # viterbi = np.zeros((input_length, len(tagset)))
    # backpointers = np.zeros((input_length, len(tagset)), dtype=int)
    
    # # Initialize the start probabilities
    # viterbi[0, :] = score('<START>', None, 0)
    
    # # Viterbi algorithm
    # for i in range(1, input_length):
    #     for j, tag in enumerate(tagset):
    #         scores = [viterbi[i-1, k] + score(tag, tagset[k], i) for k in range(len(tagset))]
    #         viterbi[i, j] = max(scores)
    #         backpointers[i, j] = np.argmax(scores)
    
    # # Backtrack to find the best sequence
    # best_path = []
    # best_tag_index = np.argmax(viterbi[input_length-1, :])
    # for i in range(input_length-1, 0, -1):
    #     best_path.insert(0, tagset[best_tag_index])
    #     best_tag_index = backpointers[i, best_tag_index]
    
    # # Return the best path, with <START> and <STOP> tags
    # return ['<START>'] + best_path + ['<STOP>']
    viterbi = [[0] * len(tagset) for _ in range(input_length)]
    backpointers = [[0] * len(tagset) for _ in range(input_length)]

    for s, tag in enumerate(tagset):
        viterbi[1][s] = score(tag, '<START>',1) 

    for i in range(2, input_length-1):
        for s, tag in enumerate(tagset):
            max_score = float('-inf')
            max_prev_tag = None
            for s_prev, prev_tag in enumerate(tagset):
                score_i = viterbi[i - 1][s_prev] + score(tag, prev_tag, i)
                if score_i > max_score:
                    max_score = score_i
                    max_prev_tag = s_prev
            viterbi[i][s] = max_score
            backpointers[i][s] = max_prev_tag

    best_seq = [''] * input_length
    max_last_score = float('-inf')
    max_last_tag = None
    for s, tag in enumerate(tagset):
        score_last = viterbi[input_length - 1][s] + score('<STOP>', tag, input_length - 1)
        if score_last > max_last_score:
            max_last_score = score_last
            max_last_tag = s

    best_seq[input_length - 1] = tagset[max_last_tag]
    current_tag = max_last_tag
    for i in range(input_length - 2, 0, -1):  
        current_tag_index = backpointers[i + 1][current_tag] 
        best_seq[i] = tagset[current_tag_index]  
        current_tag = current_tag_index  

    best_seq[0] = '<START>'
    best_seq[-1] = '<STOP>'
    return best_seq

def compute_score(tag_seq, input_length, score):
    """
    Computes the total score of a tag sequence
    :param tag_seq: Array of String of length input_length. The tag sequence including <START> and <STOP>
    :param input_length: Int. input length including the padding <START> and <STOP>
    :param score: function from current_tag (string), previous_tag (string), i (int) to the score.  i=0 points to
        <START> and i=1 points to the first token. i=input_length-1 points to <STOP>
    :return:
    """
    total_score = 0
    for i in range(1, input_length):
        total_score += score(tag_seq[i], tag_seq[i - 1], i)
    return total_score


def compute_features(tag_seq, input_length, features):
    """
    Compute f(xi, yi)
    :param tag_seq: [tags] already padded with <START> and <STOP>
    :param input_length: input length including the padding <START> and <STOP>
    :param features: func from token index to FeatureVector
    :return:
    """
    feats = FeatureVector({})
    for i in range(1, input_length):
        feats.times_plus_equal(1, features.compute_features(tag_seq[i], tag_seq[i - 1], i))
    return feats

    # Examples from class (from slides Jan 15, slide 18):
    # x = will to fight
    # y = NN TO VB
    # features(x,y) =
    #  {"wi=will^yi=NN": 1, // "wi="+current_word+"^yi="+current_tag
    # "yi-1=START^yi=NN": 1,
    # "ti=to+^yi=TO": 1,
    # "yi-1=NN+yi=TO": 1,
    # "xi=fight^yi=VB": 1,
    # "yi-1=TO^yi=VB": 1}

    # x = will to fight
    # y = NN TO VBD
    # features(x,y)=
    # {"wi=will^yi=NN": 1,
    # "yi-1=START^yi=NN": 1,
    # "ti=to+^yi=TO": 1,
    # "yi-1=NN+yi=TO": 1,
    # "xi=fight^yi=VBD": 1,
    # "yi-1=TO^yi=VBD": 1}

def sgd(training_size, epochs, gradient, parameters, training_observer):
    """
    (This should be SSGD)
    Stochastic gradient descent
    :param training_size: int. Number of examples in the training set
    :param epochs: int. Number of epochs to run SGD for
    :param gradient: func from index (int) in range(training_size) to a FeatureVector of the gradient
    :param parameters: FeatureVector.  Initial parameters.  Should be updated while training
    :param training_observer: func that takes epoch and parameters.  You can call this function at the end of each
           epoch to evaluate on a dev set and write out the model parameters for early stopping.
    :return: final parameters
    """
    # Look at the FeatureVector object.  You'll want to use the function times_plus_equal to update the
    # parameters.
    # To implement early stopping you can call the function training_observer at the end of each epoch.
    step_size = 1  # Fixed step size as mentioned
    for epoch in range(epochs):
        for i in range(training_size):
            # Calculate the gradient for the current example
            grad = gradient(i)
            # Update the parameters
            parameters.times_plus_equal(-step_size, grad)
        # Optional: Call the training observer at the end of each epoch
        # to evaluate and possibly perform early stopping
        f1_score = training_observer(epoch, parameters)
        global best_f1
        if f1_score > best_f1:
            best_f1 = f1_score
            # Save the best model
            parameters.write_to_file('best_model')
    return parameters


def adagrad(training_size, epochs, gradient, parameters, training_observer):
    """
    :param training_size: int. Number of examples in the training set
    :param epochs: int. Number of epochs to run SGD for
    :param gradient: func from index (int) in range(training_size) to a FeatureVector of the gradient
    :param parameters: FeatureVector.  Initial parameters.  Should be updated while training
    :param training_observer: func that takes epoch and parameters.  You can call this function at the end of each
           epoch to evaluate on a dev set and write out the model parameters for early stopping.
    :return: final parameters
    """
    # Look at the FeatureVector object.  You'll want to use the function times_plus_equal to update the
    # parameters.
    # To implement early stopping you can call the function training_observer at the end of each epoch.
    temp = FeatureVector({})
    best_stop_f1 = 0
    best_stop = 0

    for epoch in range(epochs):
        for i in tqdm.tqdm(range(training_size)):
            temp.power_plus_equal(2, gradient(i))
            parameters.times_plus_equal(-1, gradient(i).sqrt_div_equal(temp))
        f1 = training_observer(epoch, parameters)
        if f1 > best_stop_f1:  
            best_stop = epoch

    print(best_stop)

    return parameters
    

def train(data, feature_names, tagset, epochs):
    """
    Trains the model on the data and returns the parameters
    :param data: Array of dictionaries representing the data.  One dictionary for each data point (as created by the
        make_data_point function).
    :param feature_names: Array of Strings.  The list of feature names.
    :param tagset: Array of Strings.  The list of tags.
    :param epochs: Int. The number of epochs to train
    :return: FeatureVector. The learned parameters.
    """
    parameters = FeatureVector({})   # creates a zero vector

    def perceptron_gradient(i):
        """
        Computes the gradient of the Perceptron loss for example i
        :param i: Int
        :return: FeatureVector
        """
        inputs = data[i]
        input_len = len(inputs['tokens'])
        gold_labels = inputs['gold_tags']
        features = Features(inputs, feature_names)

        def score(cur_tag, pre_tag, i):
            return parameters.dot_product(features.compute_features(cur_tag, pre_tag, i))

        tags = decode(input_len, tagset, score)
        fvector = compute_features(tags, input_len, features)           # Add the predicted features
        #print('Input:', inputs)        # helpful for debugging
        #print("Predicted Feature Vector:", fvector.fdict)
        #print("Predicted Score:", parameters.dot_product(fvector)) # compute_score(tags, input_len, score)
        fvector.times_plus_equal(-1, compute_features(gold_labels, input_len, features))    # Subtract the features for the gold labels
        #print("Gold Labels Feature Vector: ", compute_features(gold_labels, input_len, features).fdict)
        #print("Gold Labels Score:", parameters.dot_product(compute_features(gold_labels, input_len, features)))
        return fvector

    def training_observer(epoch, parameters):
        """
        Evaluates the parameters on the development data, and writes out the parameters to a 'model.iter'+epoch and
        the predictions to 'ner.dev.out'+epoch.
        :param epoch: int.  The epoch
        :param parameters: Feature Vector.  The current parameters
        :return: Double. F1 on the development data
        """
        dev_data = read_data('ner.dev')
        (_, _, f1) = evaluate(dev_data, parameters, feature_names, tagset)

        # change output directory
        write_predictions('./output/SGD/ner.dev.out'+str(epoch), dev_data, parameters, feature_names, tagset)
        parameters.write_to_file('./output/SGD/model.iter'+str(epoch))
        return f1

    # return sgd(len(data), epochs, perceptron_gradient, parameters, training_observer)
        return adagrad(len(data), epochs, perceptron_gradient, parameters, training_observer)


def predict(inputs, input_len, parameters, feature_names, tagset):
    """
    
    :param inputs:
    :param input_len:
    :param parameters:
    :param feature_names:
    :param tagset:
    :return:
    """
    features = Features(inputs, feature_names)

    def score(cur_tag, pre_tag, i):
        return parameters.dot_product(features.compute_features(cur_tag, pre_tag, i))

    return decode(input_len, tagset, score)


def make_data_point(sent):
    """
        Creates a dictionary from String to an Array of Strings representing the data.  The dictionary items are:
        dic['tokens'] = Tokens padded with <START> and <STOP>
        dic['pos'] = POS tags padded with <START> and <STOP>
        dic['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP>
        dic['gold_tags'] = The gold tags padded with <START> and <STOP>
    :param sent: String.  The input CoNLL format string
    :return: Dict from String to Array of Strings.
    """
    dic = {}
    sent = [s.strip().split() for s in sent]
    dic['tokens'] = ['<START>'] + [s[0] for s in sent] + ['<STOP>']
    dic['pos'] = ['<START>'] + [s[1] for s in sent] + ['<STOP>']
    dic['NP_chunk'] = ['<START>'] + [s[2] for s in sent] + ['<STOP>']
    dic['gold_tags'] = ['<START>'] + [s[3] for s in sent] + ['<STOP>']
    return dic

def read_data(filename):
    """
    Reads the CoNLL 2003 data into an array of dictionaries (a dictionary for each data point).
    :param filename: String
    :return: Array of dictionaries.  Each dictionary has the format returned by the make_data_point function.
    """
    data = []
    with open(filename, 'r') as f:
        sent = []
        for line in f.readlines():
            if line.strip():
                sent.append(line)
            else:
                data.append(make_data_point(sent))
                sent = []
        data.append(make_data_point(sent))

    return data

def write_predictions(out_filename, all_inputs, parameters, feature_names, tagset):
    """
    Writes the predictions on all_inputs to out_filename, in CoNLL 2003 evaluation format.
    Each line is token, pos, NP_chuck_tag, gold_tag, predicted_tag (separated by spaces)
    Sentences are separated by a newline
    The file can be evaluated using the command: python conlleval.py < out_file
    :param out_filename: filename of the output
    :param all_inputs:
    :param parameters:
    :param feature_names:
    :param tagset:
    :return:
    """
    with open(out_filename, 'w', encoding='utf-8') as f:
        for inputs in all_inputs:
            input_len = len(inputs['tokens'])
            tag_seq = predict(inputs, input_len, parameters, feature_names, tagset)
            for i, tag in enumerate(tag_seq[1:-1]):  # deletes <START> and <STOP>
                f.write(' '.join([inputs['tokens'][i+1], inputs['pos'][i+1], inputs['NP_chunk'][i+1], inputs['gold_tags'][i+1], tag])+'\n') # i + 1 because of <START>
            f.write('\n')

def evaluate(data, parameters, feature_names, tagset):
    """
    Evaluates precision, recall, and F1 of the tagger compared to the gold standard in the data
    :param data: Array of dictionaries representing the data.  One dictionary for each data point (as created by the
        make_data_point function)
    :param parameters: FeatureVector.  The model parameters
    :param feature_names: Array of Strings.  The list of features.
    :param tagset: Array of Strings.  The list of tags.
    :return: Tuple of (prec, rec, f1)
    """
    all_gold_tags = [ ]
    all_predicted_tags = [ ]
    for inputs in data:
        all_gold_tags.extend(inputs['gold_tags'][1:-1])  # deletes <START> and <STOP>
        input_len = len(inputs['tokens'])
        all_predicted_tags.extend(predict(inputs, input_len, parameters, feature_names, tagset)[1:-1]) # deletes <START> and <STOP>
    return conllevaluate(all_gold_tags, all_predicted_tags)

def test_decoder():
    # See https://classes.soe.ucsc.edu/nlp202/Winter21/assignments/A1_Debug_Example.pdf
    
    tagset = ['NN', 'VB']     # make up our own tagset

    def score_wrap(cur_tag, pre_tag, i):
        retval = score(cur_tag, pre_tag, i)
        print('Score('+cur_tag+','+pre_tag+','+str(i)+') returning '+str(retval))
        return retval

    def score(cur_tag, pre_tag, i):
        if i == 0:
            print("ERROR: Don't call score for i = 0 (that points to <START>, with nothing before it)")
        if i == 1:
            if pre_tag != '<START>':
                print("ERROR: Previous tag should be <START> for i = 1. Previous tag = "+pre_tag)
            if cur_tag == 'NN':
                return 6
            if cur_tag == 'VB':
                return 4
        if i == 2:
            if cur_tag == 'NN' and pre_tag == 'NN':
                return 4
            if cur_tag == 'NN' and pre_tag == 'VB':
                return 9
            if cur_tag == 'VB' and pre_tag == 'NN':
                return 5
            if cur_tag == 'VB' and pre_tag == 'VB':
                return 0
        if i == 3:
            if cur_tag != '<STOP>':
                print('ERROR: Current tag at i = 3 should be <STOP>. Current tag = '+cur_tag)
            if pre_tag == 'NN':
                return 1
            if pre_tag == 'VB':
                return 1

    predicted_tag_seq = decode(4, tagset, score_wrap)
    print('Predicted tag sequence should be = <START> VB NN <STOP>')
    print('Predicted tag sequence = '+' '.join(predicted_tag_seq))
    print("Score of ['<START>','VB','NN','<STOP>'] = "+str(compute_score(['<START>','VB','NN','<STOP>'], 4, score)))
    print('Max score should be = 14')
    print('Max score = '+str(compute_score(predicted_tag_seq, 4, score)))


def main_predict(data_filename, model_filename):
    """
    Main function to make predictions.
    Loads the model file and runs the NER tagger on the data, writing the output in CoNLL 2003 evaluation format to data_filename.out
    :param data_filename: String
    :param model_filename: String
    :return: None
    """
    data = read_data(data_filename)
    parameters = FeatureVector({})
    parameters.read_from_file(model_filename)

    tagset = ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'I-ORG', 'I-MISC', 'O']
    feature_names = ['tag', 'prev_tag', 'current_word','len_k','gazetteer','capi']

    write_predictions('./output/SGD/'data_filename+'.out', data, parameters, feature_names, tagset)
    evaluate(data, parameters, feature_names, tagset)

    return


def main_train():
    """
    Main function to train the model
    :return: None
    """
    print('Reading training data')
    train_data = read_data('ner.train')
    #train_data = read_data('ner.train')[1:1] # if you want to train on just one example

    tagset = ['B-PER', 'B-LOC', 'B-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'I-ORG', 'I-MISC', 'O']
    feature_names = ['tag', 'prev_tag', 'current_word','len_k','gazetteer','capi']

    print('Training...')
    parameters = train(train_data, feature_names, tagset, epochs=10)
    print('Training done')
    dev_data = read_data('ner.dev')
    evaluate(dev_data, parameters, feature_names, tagset)
    test_data = read_data('ner.test')
    evaluate(test_data, parameters, feature_names, tagset)
    parameters.write_to_file('model_sgd')

    return


class Features(object):
    def __init__(self, inputs, feature_names):
        """
        Creates a Features object
        :param inputs: Dictionary from String to an Array of Strings.
            Created in the make_data_point function.
            inputs['tokens'] = Tokens padded with <START> and <STOP>
            inputs['pos'] = POS tags padded with <START> and <STOP>
            inputs['NP_chunk'] = Tags indicating noun phrase chunks, padded with <START> and <STOP>
            inputs['gold_tags'] = DON'T USE! The gold tags padded with <START> and <STOP>
        :param feature_names: Array of Strings.  The list of features to compute.
        """
        self.feature_names = feature_names
        self.inputs = inputs

        self.gazetteer = gazetteer

    def compute_features(self, cur_tag, pre_tag, i):
        """
        Computes the local features for the current tag, the previous tag, and position i
        :param cur_tag: String.  The current tag.
        :param pre_tag: String.  The previous tag.
        :param i: Int. The position
        :return: FeatureVector
        """
        token_i=self.inputs['tokens'][i]
        feats = FeatureVector({})
        # if 'tag' in self.feature_names:
        #     feats.times_plus_equal(1, FeatureVector({'t='+cur_tag: 1}))
        # if 'prev_tag' in self.feature_names:
        #     feats.times_plus_equal(1, FeatureVector({'ti='+cur_tag+"+ti-1="+pre_tag: 1}))
        # if 'current_word' in self.feature_names:
        #     feats.times_plus_equal(1, FeatureVector({f't='+cur_tag+'+w='+self.inputs['tokens'][i]: 1}))
        if 'tag' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({f't={cur_tag}': 1}))
        if 'prev_tag' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({f'ti={cur_tag}+ti-1={pre_tag}': 1}))
        if 'current_word' in self.feature_names:
            feats.times_plus_equal(1, FeatureVector({f't={cur_tag}+w={token_i}': 1}))
        if 'shape' in self.feature_names:
            shape = ''.join(['A' if c.isupper() else 'a' if c.islower() else 'd' if c.isdigit() else c for c in self.inputs['tokens']])
            feats.times_plus_equal(1, FeatureVector({f'si={shape}+ti={cur_tag}': 1.0}))
        # for k in range(1, 5):  # Prefixes of length 1 to 4
        #     if len(self.inputs['tokens']) >= k:
        #         prefix = self.inputs['tokens'][:k]
        #         feats.times_plus_equal(1, FeatureVector({f'PRE{k}i={prefix}+ti={cur_tag}': 1.0}))
        if 'len_k' in self.feature_names:
            for k in range(len(self.inputs['tokens'][i][:4])):
                feats.times_plus_equal(1, FeatureVector({'prei='+self.inputs['tokens'][i][:k]+'+t='+cur_tag: 1}))
        # if 'gazetteer' in self.feature_names and self.inputs['tokens'][i] in gazetteer and gazetteer[self.inputs['tokens'][i]] == cur_tag.split('-')[1]:
        #     feats.times_plus_equal(1, FeatureVector({f'GAZ=True+t={cur_tag}': 1.0}))
        if 'gazetteer' in self.feature_names:
            if self.inputs['tokens'][i] in self.gazetteer.keys() and self.gazetteer[self.inputs['tokens'][i]] == cur_tag: #'LOC': #and has LOC tag:
                feats.times_plus_equal(1, FeatureVector({'gazi='+"True"+'+t='+cur_tag: 1}))
        # if 'capitalization' in self.feature_names and self.inputs['tokens'][0].isupper():
        #     feats.times_plus_equal(1, FeatureVector({f'CAPi=True+ti={cur_tag}': 1.0}))
        #     # feats.times_plus_equal(1, FeatureVector({'capi='+"True"+"+ti="+cur_tag: 1}))
        if 'capi' in self.feature_names:
            if self.inputs['tokens'][i][0].isupper():
                feats.times_plus_equal(1, FeatureVector({'capi='+"True"+'+t='+cur_tag: 1}))

        return feats

class FeatureVector(object):

    def __init__(self, fdict):
        self.fdict = fdict

    def times_plus_equal(self, scalar, v2):
        """
        self += scalar * v2
        :param scalar: Double
        :param v2: FeatureVector
        :return: None
        """
        for key, value in v2.fdict.items():
            self.fdict[key] = scalar * value + self.fdict.get(key, 0)


    def dot_product(self, v2):
        """
        Computes the dot product between self and v2.  It is more efficient for v2 to be the smaller vector (fewer
        non-zero entries).
        :param v2: FeatureVector
        :return: Int
        """
        retval = 0
        for key, value in v2.fdict.items():
            retval += value * self.fdict.get(key, 0)
        return retval

    def write_to_file(self, filename):
        """
        Writes the feature vector to a file.
        :param filename: String
        :return: None
        """
        print('Writing to ' + filename)
        with open(filename, 'w', encoding='utf-8') as f:
            for key, value in self.fdict.items():
                f.write('{} {}\n'.format(key, value))


    def read_from_file(self, filename):
        """
        Reads a feature vector from a file.
        :param filename: String
        :return: None
        """
        self.fdict = {}
        with open(filename, 'r') as f:
            for line in f.readlines():
                txt = line.split()
                self.fdict[txt[0]] = float(txt[1])

# best_f1=0
main_train()    # Uncomment to train a model (need to implement 'sgd' function)
main_predict('ner.dev', 'model')  # Uncomment to predict on 'dev.ner' using the model 'model' (need to implement 'decode' function)



Reading training data
Training...
Training done


AttributeError: 'NoneType' object has no attribute 'dot_product'

In [22]:
main_predict('ner.dev', 'model')

processed 51578 tokens with 5917 phrases; found: 7463 phrases; correct: 2062.
accuracy:  40.15%; (non-O)
accuracy:  82.89%; precision:  27.63%; recall:  34.85%; FB1:  30.82
              LOC: precision:  64.17%; recall:  54.32%; FB1:  58.83  1549
             MISC: precision:  47.11%; recall:  33.92%; FB1:  39.44  658
              ORG: precision:  55.56%; recall:  17.52%; FB1:  26.64  423
              PER: precision:  10.82%; recall:  28.55%; FB1:  15.69  4833


In [23]:
main_predict('ner.test', 'model')

processed 46666 tokens with 5616 phrases; found: 7439 phrases; correct: 1654.
accuracy:  35.74%; (non-O)
accuracy:  80.17%; precision:  22.23%; recall:  29.45%; FB1:  25.34
              LOC: precision:  59.45%; recall:  52.88%; FB1:  55.97  1482
             MISC: precision:  30.40%; recall:  25.11%; FB1:  27.50  579
              ORG: precision:  50.52%; recall:  11.90%; FB1:  19.26  388
              PER: precision:   8.04%; recall:  25.03%; FB1:  12.17  4990


In [15]:
best_f1

30.822122571001497

To sort the model weights for easy viewing, you can use Unix commands:

In [ ]:
!cat "/content/gdrive/My Drive/NLP-202-A1-Code/model" | awk '{print $2, $1}' | sort -gr > "/content/gdrive/My Drive/NLP-202-A1-Code/model.sorted.txt"

The file `model.sorted.txt` will be viewable in your Google Drive folder.